In [2]:
%pip install pandas

In [3]:
%pip install scipy

In [7]:
%pip install ipywidgets

In [10]:
import unittest
import pandas as pd
import numpy as np
from io import StringIO

def load_goals_from_csv(file_like):
    return pd.read_csv(file_like)

def select_top_communicative_goals(weighted_goals, reweighted_goals, top_n=3):
    combined_goals = pd.merge(weighted_goals, reweighted_goals, on='goal', how='outer')
    combined_goals['final_weight'] = (
        combined_goals['weight'].fillna(0) + combined_goals['reweight'].fillna(0)
    )
    # Приводим тип к float для корректной работы nlargest
    combined_goals['final_weight'] = combined_goals['final_weight'].astype(float)
    top_goals = combined_goals.nlargest(top_n, 'final_weight')
    return top_goals

class TestGoalFunctions(unittest.TestCase):

    def setUp(self):
        # Набор данных 1
        self.weighted_goals_csv_1 = StringIO("""
goal,weight,duration
Goal 1,5,10
Goal 2,3,15
Goal 3,4,5
Goal 4,2,20
""")
        self.reweighted_goals_csv_1 = StringIO("""
goal,reweight
Goal 1,1
Goal 2,2
Goal 3,0
Goal 4,3
""")

        # Набор данных 2
        self.weighted_goals_csv_2 = StringIO("""
goal,weight,duration
Goal A,7,12
Goal B,6,18
Goal C,5,8
Goal D,4,25
""")
        self.reweighted_goals_csv_2 = StringIO("""
goal,reweight
Goal A,2
Goal B,1
Goal C,3
Goal D,0
""")

        # Набор данных 3
        self.weighted_goals_csv_3 = StringIO("""
goal,weight,duration
Goal X,10,30
Goal Y,8,14
Goal Z,6,22
Goal W,4,10
""")
        self.reweighted_goals_csv_3 = StringIO("""
goal,reweight
Goal X,5
Goal Y,2
Goal Z,1
Goal W,3
""")

        # Набор данных 4
        self.weighted_goals_csv_4 = StringIO("""
goal,weight,duration
Task 1,9,15
Task 2,8,10
Task 3,7,20
Task 4,6,25
""")
        self.reweighted_goals_csv_4 = StringIO("""
goal,reweight
Task 1,4
Task 2,3
Task 3,2
Task 4,1
""")

        # Набор данных 5
        self.weighted_goals_csv_5 = StringIO("""
goal,weight,duration
Aim 1,12,18
Aim 2,11,16
Aim 3,10,14
Aim 4,9,12
""")
        self.reweighted_goals_csv_5 = StringIO("""
goal,reweight
Aim 1,6
Aim 2,5
Aim 3,4
Aim 4,3
""")

        # Список наборов данных для тестирования
        self.data_sets = [
            (self.weighted_goals_csv_1, self.reweighted_goals_csv_1),
            (self.weighted_goals_csv_2, self.reweighted_goals_csv_2),
            (self.weighted_goals_csv_3, self.reweighted_goals_csv_3),
            (self.weighted_goals_csv_4, self.reweighted_goals_csv_4),
            (self.weighted_goals_csv_5, self.reweighted_goals_csv_5),
        ]

    def test_load_goals_from_csv(self):
        for weighted_csv, reweighted_csv in self.data_sets:
            weighted_csv.seek(0)
            df = load_goals_from_csv(weighted_csv)
            self.assertEqual(len(df), 4, "Должно быть загружено 4 цели.")
            self.assertIn('goal', df.columns, "Столбец 'goal' должен присутствовать.")

    def test_select_top_communicative_goals(self):
        for weighted_csv, reweighted_csv in self.data_sets:
            weighted_csv.seek(0)
            reweighted_csv.seek(0)
            weighted_goals = pd.read_csv(weighted_csv)
            reweighted_goals = pd.read_csv(reweighted_csv)
            top_goals = select_top_communicative_goals(weighted_goals, reweighted_goals)
            self.assertEqual(len(top_goals), 3, "Должно быть выбрано 3 топ цели.")
            self.assertIn('final_weight', top_goals.columns, "Столбец 'final_weight' должен присутствовать.")

    def test_select_top_communicative_goals_no_data(self):
        empty_weighted = pd.DataFrame(columns=['goal', 'weight', 'duration'])
        empty_reweighted = pd.DataFrame(columns=['goal', 'reweight'])
        top_goals = select_top_communicative_goals(empty_weighted, empty_reweighted)
        self.assertTrue(top_goals.empty, "Результат должен быть пустым при отсутствии данных.")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

    print("Все тесты прошли успешно!")


...
----------------------------------------------------------------------
Ran 3 tests in 0.081s

OK


Все тесты прошли успешно!
